In [10]:
def WriteHeader(f, TaskName):
    f.write("#!/bin/bash\n\n")
    f.write("#SBATCH --partition=multiple\n")
    f.write("#SBATCH --nodes=2\n")
    f.write("#SBATCH --ntasks-per-node=40\n")
    f.write("#SBATCH --mem-per-cpu=1125mb\n")
    f.write("#SBATCH --time=72:00:00\n")
    f.write(f"#SBATCH --job-name={TaskName}\n")
    f.write("#SBATCH --error=%x.%j.err\n")
    f.write("#SBATCH --output=%x.%j.out\n")
    f.write("#SBATCH --mail-user=hzhao@teco.edu\n\n")

In [4]:
with open(f'run_baseline_pNN.sh', 'w') as f:
    for ds in range(13):
        line = f'sbatch experiment.py --DATASET {ds} --skipconnection False --areaestimator none  --projectname Baseline\n'
        f.write(line)

In [13]:
for epsilon in [0., 0.05, 0.1]:
    with open(f'run_EA_independent_{epsilon}.sh', 'w') as f:
        WriteHeader(f, f'EAI{epsilon}')
        for ds in range(13):
            for seed in range(10):
                line = f'python3 experiment_EA.py --DATASET {ds:02d} --SEED {seed:02d} --e_train {epsilon} --projectname VariationAware_EA_{epsilon}\n'
                f.write(line)
        
        f.write("\nwait\n")

In [15]:
for epsilon in [0., 0.05, 0.1]:
    with open(f'run_EA_shared_{epsilon}.sh', 'w') as f:
        WriteHeader(f, f'EAS{epsilon}')
        for ds in range(13):
            for seed in range(10):
                line = f'python3 experiment_EA_shared.py --DATASET {ds:02d} --SEED {seed:02d} --e_train {epsilon} --projectname VariationAware_EA_{epsilon}_shared\n'
                f.write(line)
        
        f.write("\nwait\n")

In [1]:
with open('evaluation_baseline_area.sh', 'w') as f:
    for ds in range(13):
        line = f'sbatch evaluation.py --DATASET {ds}  --areaestimator none --DEVICE cpu\n'
        f.write(line)

In [2]:
gammas = np.linspace(0,1,50)
gammas = np.round(gammas, 5)
gammas = np.unique(gammas)

In [3]:
gammas

array([0.     , 0.02041, 0.04082, 0.06122, 0.08163, 0.10204, 0.12245,
       0.14286, 0.16327, 0.18367, 0.20408, 0.22449, 0.2449 , 0.26531,
       0.28571, 0.30612, 0.32653, 0.34694, 0.36735, 0.38776, 0.40816,
       0.42857, 0.44898, 0.46939, 0.4898 , 0.5102 , 0.53061, 0.55102,
       0.57143, 0.59184, 0.61224, 0.63265, 0.65306, 0.67347, 0.69388,
       0.71429, 0.73469, 0.7551 , 0.77551, 0.79592, 0.81633, 0.83673,
       0.85714, 0.87755, 0.89796, 0.91837, 0.93878, 0.95918, 0.97959,
       1.     ])

In [2]:
with open('run_baseline_pNN.sh', 'w') as f:
    # Writing the SBATCH parameters first
    f.write("#!/bin/bash\n\n")
    f.write("#SBATCH --partition=multiple\n")
    f.write("#SBATCH --nodes=2\n")
    f.write("#SBATCH --ntasks-per-node=40\n")
    f.write("#SBATCH --mem-per-cpu=1125mb\n")
    f.write("#SBATCH --time=72:00:00\n")
    f.write("#SBATCH --job-name=batch\n")
    f.write("#SBATCH --error=%x.%j.err\n")
    f.write("#SBATCH --output=%x.%j.out\n")
    f.write("#SBATCH --mail-user=hzhao@teco.edu\n\n")

    # Looping through the dataset values and writing the commands
    for ds in range(5):  # Adjust the range if needed
        script_name = f"experiment_{ds}.py"
        line = f'python3 {script_name} {ds + 1} &\n'
        f.write(line)

    f.write("\nwait\n")
